In [207]:
import pandas as pd

In [208]:
df = pd.read_csv('03_Netprofit_final_eda.csv')

In [209]:
df.head()

,Order date,Exchange Rate,Order number,Item Title,Tracking Number,Category,Item Price,Earning,Shipping Price,Custom Tax,Purchase Cost,Net profit,Earning_W
0,2025-08-14,1383.80,07-13453-25016,Marc_Jacobs wool blend burgundy color skirt,883706302029,Clothes,114.23,96.24,27140,0,35251.28,70785.63,133176.91
1,2025-08-15,1383.80,22-13439-84321,Coach cignature Silver Belt Killer Bee Ornamen...,883735973984,Accessaries,107.31,90.34,40710,0,30001.01,54301.49,125012.49
2,2025-08-16,1383.80,06-13463-90074,Marc_Jacobs C391003 SUSAN purple color 2WAY,883706233389,Bags,161.92,136.05,39550,0,71434.14,77281.85,188265.99
3,2025-08-18,1382.20,21-13449-06816,Burberry Black Leather Multi Embellished Prosu...,883766262737,Bags,582.31,490.30,60110,0,390388.10,227194.56,677692.66
4,2025-08-25,1395.60,03-13505-86833,Fendi 8BR589 Black Leather Spy Mini Tote,883920026391,Bags,565.00,478.06,43560,0,377254.84,246365.70,667180.54


In [210]:
df['Item Price W'] = df['Item Price']*1470

In [211]:
df.head()

,Order date,Exchange Rate,Order number,Item Title,Tracking Number,Category,Item Price,Earning,Shipping Price,Custom Tax,Purchase Cost,Net profit,Earning_W,Item Price W
0,2025-08-14,1383.80,07-13453-25016,Marc_Jacobs wool blend burgundy color skirt,883706302029,Clothes,114.23,96.24,27140,0,35251.28,70785.63,133176.91,167918.10
1,2025-08-15,1383.80,22-13439-84321,Coach cignature Silver Belt Killer Bee Ornamen...,883735973984,Accessaries,107.31,90.34,40710,0,30001.01,54301.49,125012.49,157745.70
2,2025-08-16,1383.80,06-13463-90074,Marc_Jacobs C391003 SUSAN purple color 2WAY,883706233389,Bags,161.92,136.05,39550,0,71434.14,77281.85,188265.99,238022.40
3,2025-08-18,1382.20,21-13449-06816,Burberry Black Leather Multi Embellished Prosu...,883766262737,Bags,582.31,490.30,60110,0,390388.10,227194.56,677692.66,855995.70
4,2025-08-25,1395.60,03-13505-86833,Fendi 8BR589 Black Leather Spy Mini Tote,883920026391,Bags,565.00,478.06,43560,0,377254.84,246365.70,667180.54,830550.00


In [212]:
df['Order date'] = pd.to_datetime(df['Order date'])
df['Month'] = df['Order date'].dt.month

In [213]:

# 환불 비용 분석 (Month별)
# 가정 1: 전체 상품의 10%가 환불됨
# 가정 2: 환불 상품은 Item Price의 50%로 재판매 → 실질 환불 비용 = 환불 수량 × Item Price × 50%

REFUND_RATE = 0.10   # 환불율 10%
RESALE_RATE = 0.50   # 재판매 가격 비율 50%
EXCHANGE_RATE = 1470 # 환율

# Month별 상품 수 & Item Price 통계
monthly_count = df.groupby('Month')['Item Price'].count()
monthly_price = df.groupby('Month')['Item Price'].agg(['min', 'max', 'mean']).round(2)

# 환불 수량 = 상품 수 × 10%
monthly_stats = monthly_price.copy()
monthly_stats.insert(0, '상품 수', monthly_count)
monthly_stats['환불 수량 (10%)'] = (monthly_stats['상품 수'] * REFUND_RATE).apply(lambda x: round(x, 1))

# 환불 비용 = 환불 수량 × Item Price × 50% (KRW 환산)
monthly_stats['최저가_환불비용 (KRW)'] = (monthly_stats['환불 수량 (10%)'] * monthly_stats['min'] * RESALE_RATE * EXCHANGE_RATE).round(0).astype(int)
monthly_stats['최고가_환불비용 (KRW)'] = (monthly_stats['환불 수량 (10%)'] * monthly_stats['max'] * RESALE_RATE * EXCHANGE_RATE).round(0).astype(int)
monthly_stats['평균가_환불비용 (KRW)'] = (monthly_stats['환불 수량 (10%)'] * monthly_stats['mean'] * RESALE_RATE * EXCHANGE_RATE).round(0).astype(int)

monthly_stats.columns = ['상품 수', '최저 Item Price (USD)', '최고 Item Price (USD)', '평균 Item Price (USD)',
                         '환불 수량 (10%)', '최저가_환불비용 (KRW)', '최고가_환불비용 (KRW)', '평균가_환불비용 (KRW)']

# 천 단위 콤마 포맷
display_df = monthly_stats.copy()
krw_cols = ['최저가_환불비용 (KRW)', '최고가_환불비용 (KRW)', '평균가_환불비용 (KRW)']
display_df[krw_cols] = display_df[krw_cols].map(lambda x: f"{x:,}")

print("=== Month별 환불 비용 분석 ===")
print("(환불율 10% | 재판매 가격 50% 가정 | 환율 1,470원)")
print()
display_df


=== Month별 환불 비용 분석 ===
(환불율 10% | 재판매 가격 50% 가정 | 환율 1,470원)



,상품 수,최저 Item Price (USD),최고 Item Price (USD),평균 Item Price (USD),환불 수량 (10%),최저가_환불비용 (KRW),최고가_환불비용 (KRW),평균가_환불비용 (KRW)
Month,,,,,,,,
1,10,193.86,1169.29,476.74,1.00,"142,487","859,428","350,404"
8,6,107.31,582.31,338.91,0.60,"47,324","256,799","149,459"
9,14,103.85,9684.62,1317.56,1.40,"106,862","9,965,474","1,355,769"
10,19,86.54,7720.00,715.99,1.90,"120,853","10,780,980","999,880"
11,8,214.43,549.35,400.36,0.80,"126,085","323,018","235,412"
12,9,95.00,3541.43,880.84,0.90,"62,842","2,342,656","582,676"


In [214]:
MA = df.groupby('Month')['Item Price'].agg(['min', 'max', 'mean'])
MA = (MA*1470).round(2)
MA.columns = ['Item Price(Min)', 'Item Price(Max)', 'Item Price(Mean)']

def second_largest(x):
    unique_vals = x.nlargest(2)
    return unique_vals.iloc[1]
def second_smallest(x):
    unique_vals = x.nsmallest(2)
    return unique_vals.iloc[1]

second_MA = df.groupby('Month')['Item Price'].agg(
    Second_Min=second_smallest,
    Second_Max=second_largest
)

MA_results = pd.merge(MA, second_MA, how='inner', on='Month')

MA_results['Second_Min'] = (MA_results['Second_Min']*1470).round(2)
MA_results['Second_Max'] = (MA_results['Second_Max']*1470).round(2)

MA_results['Item Count'] = df.groupby('Month')['Order number'].count()


In [215]:
MA_results['Min_refund_cost'] = MA_results.apply(
    lambda row: (row['Item Price(Min)'] + row['Second_Min']) * 0.5
               if round(row['Item Count'] * 0.1) >= 2
               else row['Item Price(Min)'] * 0.5,
    axis=1
).round(0).astype(int)

MA_results['Max_refund_cost'] = MA_results.apply(
    lambda row: (row['Item Price(Max)'] + row['Second_Max']) * 0.5
               if round(row['Item Count'] * 0.1) >= 2
               else row['Item Price(Max)'] * 0.5,
    axis=1
).round(0).astype(int)

MA_results['Mean_refund_cost'] = (MA_results['Item Price(Mean)'] * 0.5).round(0).astype(int)

MA_results.style.format("{:,.0f}")

,Item Price(Min),Item Price(Max),Item Price(Mean),Second_Min,Second_Max,Item Count,Min_refund_cost,Max_refund_cost,Mean_refund_cost
Month,,,,,,,,,
1,"284,974","1,718,856","700,812","326,546","1,598,096",10,"142,487","859,428","350,406"
8,"157,746","855,996","498,198","167,918","830,550",6,"78,873","427,998","249,099"
9,"152,660","14,236,391","1,936,818","231,158","3,883,064",14,"76,330","7,118,196","968,409"
10,"127,214","11,348,400","1,052,509","162,832","1,414,596",19,"145,023","6,381,498","526,255"
11,"315,212","807,544","588,529","470,356","768,604",8,"157,606","403,772","294,265"
12,"139,650","5,205,902","1,294,836","381,362","1,409,098",8,"69,825","2,602,951","647,418"


In [216]:
MA_refund = MA_results[['Min_refund_cost', 'Max_refund_cost', 'Mean_refund_cost']]

MA_refund.style.format("{:,.0f}")

,Min_refund_cost,Max_refund_cost,Mean_refund_cost
Month,,,
1,"142,487","859,428","350,406"
8,"78,873","427,998","249,099"
9,"76,330","7,118,196","968,409"
10,"145,023","6,381,498","526,255"
11,"157,606","403,772","294,265"
12,"69,825","2,602,951","647,418"


In [217]:
df.head(10)

,Order date,Exchange Rate,Order number,Item Title,Tracking Number,Category,Item Price,Earning,Shipping Price,Custom Tax,Purchase Cost,Net profit,Earning_W,Item Price W,Month
0,2025-08-14,1383.80,07-13453-25016,Marc_Jacobs wool blend burgundy color skirt,883706302029,Clothes,114.23,96.24,27140,0,35251.28,70785.63,133176.91,167918.10,8
1,2025-08-15,1383.80,22-13439-84321,Coach cignature Silver Belt Killer Bee Ornamen...,883735973984,Accessaries,107.31,90.34,40710,0,30001.01,54301.49,125012.49,157745.70,8
2,2025-08-16,1383.80,06-13463-90074,Marc_Jacobs C391003 SUSAN purple color 2WAY,883706233389,Bags,161.92,136.05,39550,0,71434.14,77281.85,188265.99,238022.40,8
3,2025-08-18,1382.20,21-13449-06816,Burberry Black Leather Multi Embellished Prosu...,883766262737,Bags,582.31,490.30,60110,0,390388.10,227194.56,677692.66,855995.70,8
4,2025-08-25,1395.60,03-13505-86833,Fendi 8BR589 Black Leather Spy Mini Tote,883920026391,Bags,565.00,478.06,43560,0,377254.84,246365.70,667180.54,830550.00,8
5,2025-08-30,1388.60,17-13506-04539,MULBERRY HH5988 Bayswater CHOCOLATE COLOR M TO...,884125049013,Bags,502.69,423.32,75090,0,329979.64,182752.51,587822.15,738954.30,8
6,2025-09-01,1387.70,01-13538-12810,Louis Vuitton M95737 Joke Man Crazy Showline Tote,884124998903,Bags,935.77,792.04,78010,0,658561.63,362542.28,1099113.91,1375581.90,9
7,2025-09-02,1392.40,04-13536-34068,VALENTINO QW1B0809 Navy Leather Rockstud Small...,884174761629,Bags,449.62,380.19,51100,0,289714.92,188561.64,529376.56,660941.40,9
8,2025-09-03,1392.20,03-13542-76443,VALENTINO WW2B0123NAP indie pink color rockstu...,884213307003,Bags,795.00,679.72,40760,0,551758.06,353788.12,946306.18,1168650.00,9
9,2025-09-03,1392.20,17-13525-95052,Chloe 7EPM09 - 7E422 Paddington Brown Ladder Z...,884246280770,Bags,103.85,88.75,38390,0,27375.87,57791.88,123557.75,152659.50,9


In [218]:
df['eBay Cost'] = df['Item Price W'] - df['Earning_W']

df_results = pd.merge(df, MA_refund, how='left', on='Month')

In [219]:
df_month = df.groupby('Month')[['Item Price W', 'Shipping Price', 'Custom Tax', 'Purchase Cost', 'eBay Cost']].agg(['sum'])

df_month.columns = df_month.columns.droplevel(1)

df_final = pd.merge(df_month, MA_refund, how='inner', on='Month')

df_final = df_final.reset_index()
df_final.to_csv('04_checkrefund_eda.csv', index=False)

In [220]:
improve_rate = 2.2
df_final['Improve Price'] = (df_final['Item Price W']/2*improve_rate)

base = (df_final['Improve Price'] 
        - df_final['Shipping Price'] 
        - df_final['Custom Tax'] 
        - df_final['Purchase Cost'] 
        - df_final['eBay Cost'])

for suffix in ['Min', 'Max', 'Mean']:
    df_final[f'Net Profit with Refund({suffix})'] = base - df_final[f'{suffix}_refund_cost']

In [221]:
pd.options.display.float_format = '{:.2f}'.format

df_final.style.format("{:,.0f}")

,Month,Item Price W,Shipping Price,Custom Tax,Purchase Cost,eBay Cost,Min_refund_cost,Max_refund_cost,Mean_refund_cost,Improve Price,Net Profit with Refund(Min),Net Profit with Refund(Max),Net Profit with Refund(Mean)
0,1,"7,008,122","605,900","1,898,030","3,102,934","751,527","142,487","859,428","350,406","7,708,934","1,208,056","491,115","1,000,137"
1,8,"2,989,186","286,160",0,"1,234,309","610,035","78,873","427,998","249,099","3,288,105","1,078,727","729,602","908,501"
2,9,"27,115,458","895,450","2,374,710","13,275,249","4,238,928","76,330","7,118,196","968,409","29,827,004","8,966,336","1,924,470","8,074,257"
3,10,"19,997,674","1,138,800","4,942,630","9,344,474","3,292,429","145,023","6,381,498","526,255","21,997,442","3,134,086","-3,102,389","2,752,854"
4,11,"4,708,234","572,980","1,400,140","2,018,727","656,423","157,606","403,772","294,265","5,179,057","373,180","127,014","236,521"
5,12,"11,653,528","588,830","2,675,290","5,551,979","1,055,407","69,825","2,602,951","647,418","12,818,881","2,877,549","344,423","2,299,956"
